In [1]:
import gymnasium as gym
import numpy as np
import torch
import open3d as o3d
from gymnasium import spaces
import copy
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:
class CoverageEnv(gym.Env):
    def __init__(self, obj_file_path, radius=20.0, coverage_threshold=0.98):
        super(CoverageEnv, self).__init__()

        self.mesh = o3d.io.read_triangle_mesh(obj_file_path)
        self.mesh.compute_vertex_normals()
        
        self.vertices = torch.tensor(np.asarray(self.mesh.vertices), dtype=torch.float32).cuda()
        self.faces = torch.tensor(np.asarray(self.mesh.triangles), dtype=torch.int64).cuda()
        self.normals = torch.tensor(np.asarray(self.mesh.vertex_normals), dtype=torch.float32).cuda()
        
        self.num_faces = self.faces.shape[0]
        self.radius = radius
        self.coverage_threshold = coverage_threshold

        self.observation_space = spaces.Box(low=0, high=1, shape=(self.num_faces,), dtype=np.int8)
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)

        self.covered_faces = torch.zeros(self.num_faces, dtype=torch.bool).cuda()
        self.total_covered = 0
        self.steps = 0

        # self.vis = o3d.visualization.Visualizer()
        # self.vis.create_window(visible=False)
        # self.vis.add_geometry(self.mesh)
        # self.ctr = self.vis.get_view_control()
        # self.camera = self.vis.get_view_control().convert_to_pinhole_camera_parameters()
        self.actions = []

        self.reward_weights = np.array([1,     # w_newly_covered, 
                                       100.0,    # w_threshold_bonus, 
                                       -10.0,    # w_step_penalty, 
                                    ])

    def reset(self, seed=None):
        super().reset(seed=seed)
        self.covered_faces.fill_(False)
        self.total_covered = 0
        self.steps = 0
        return self._get_obs(), {}

    def step(self, action):
        self.last_action = action
        self.actions.append(action)
        self.steps += 1
        
        action = torch.tensor(action, dtype=torch.float32).cuda()
        theta = (action[0] + 1) * np.pi
        phi = action[1] * np.pi / 2

        x = self.radius * torch.sin(theta) * torch.cos(phi)
        y = self.radius * torch.sin(theta) * torch.sin(phi)
        z = self.radius * torch.cos(theta)
        viewpoint = torch.tensor([x, y, z], dtype=torch.float32).cuda()

        face_centers = torch.mean(self.vertices[self.np.dot(self.reward_weights, r)faces], dim=1)
        face_normals = torch.cross(self.vertices[self.faces[:, 1]] - self.vertices[self.faces[:, 0]],
                                   self.vertices[self.faces[:, 2]] - self.vertices[self.faces[:, 0]])
        face_normals = face_normals / torch.norm(face_normals, dim=1, keepdim=True)

        view_directions = face_centers - viewpoint
        view_directions = view_directions / torch.norm(view_directions, dim=1, keepdim=True)

        visible_faces = torch.sum(face_normals * view_directions, dim=1) < 0

        newly_covered = visible_faces & ~self.covered_faces
        self.covered_faces |= visible_faces
        self.total_covered = torch.sum(self.covered_faces).item()

        coverage = self.total_covered / self.num_faces
        reward = self._calculate_reward(newly_covered, coverage)
        done = coverage >= self.coverage_threshold or self.steps >= 1000

        return self._get_obs(), reward, done, False, {}

    def _calculate_reward(self, newly_covered, coverage):
        # Calculate individual reward components
        r = np.array([
            torch.sum(newly_covered).item(),
            1 if coverage >= self.coverage_threshold else 0,
            len(self.actions)
            ])
        print(r)
        print(np.dot(self.reward_weights, r))
        # Combine reward components
        return np.dot(self.reward_weights, r)


    def _get_obs(self):
        return self.covered_faces.cpu().numpy()

    def render(self):
        # Create a new visualizer for each render call
        vis = o3d.visualization.Visualizer()
        vis.create_window()

        # Create a copy of the mesh for visualization
        vis_mesh = o3d.geometry.TriangleMesh()
        vis_mesh.vertices = o3d.utility.Vector3dVector(self.vertices.cpu().numpy())
        vis_mesh.triangles = o3d.utility.Vector3iVector(self.faces.cpu().numpy())

        # Color the mesh based on coverage
        colors = np.array([[0, 1, 0] if covered else [1, 0, 0] for covered in self.covered_faces.cpu().numpy()])
        vis_mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

        # Add the mesh to the visualizer
        vis.add_geometry(vis_mesh)

        # Create a sphere to represent the viewpoint
        viewpoint = o3d.geometry.TriangleMesh.create_sphere(radius=1)
        viewpoint.compute_vertex_normals()
        viewpoint.paint_uniform_color([1, 0, 0])  # Blue color for viewpoint

        # Set the position of the viewpoint
        # Move last_action to CPU before using with numpy
        last_action_cpu = self.last_action.cpu().numpy()
        theta = (last_action_cpu[0] + 1) * np.pi
        phi = last_action_cpu[1] * np.pi / 2
        x = self.radius * np.sin(theta) * np.cos(phi)
        y = self.radius * np.sin(theta) * np.sin(phi)
        z = self.radius * np.cos(theta)
        viewpoint.translate([x, y, z])

        # # Add the viewpoint to the visualizer
        vis.add_geometry(viewpoint)
        # o3d.visualization.draw_geometries([viewpoint, vis_mesh])

        # # Set up the camera view
        ctr = vis.get_view_control()
        ctr.set_front([0, 0, -1])
        ctr.set_lookat([0, 0, 0])
        ctr.set_up([0, 1, 0])
        ctr.set_zoom(0.8)

        # # Update the geometry and render
        vis.update_geometry(vis_mesh)
        vis.update_geometry(viewpoint)
        vis.poll_events()
        vis.update_renderer()

        # Capture and display the image
        img = vis.capture_screen_float_buffer(do_render=True)
        vis.destroy_window()

        # Convert the image to numpy array and display using matplotlib
        plt.imshow(np.asarray(img))
        plt.title(f"Coverage: {self.total_covered / self.num_faces:.2%}")
        plt.axis('off')
        plt.show()


    def close(self):
        if hasattr(self, 'vis'):
            self.vis.destroy_window()

In [10]:
env = CoverageEnv("/home/dir/RL_CoveragePlanning/test_models/modified/test_7.obj")
obs = env.reset()

for i in range(20):
    action = env.action_space.sample()
    action_tensor = torch.tensor(action, dtype=torch.float32).cuda()
    
    obs, reward, done, _, info = env.step(action_tensor)
    print(f"Idx: {i:3d} | reward: {reward:7.2f} | Coverage Progress: {env.total_covered / env.num_faces :5.4f} |Done: {done}")
    # env.render()
    
    if done:
        break

[4551    0    1]
4541.0
Idx:   0 | reward: 4541.00 | Coverage Progress: 0.4551 |Done: False
[2406    0    2]
2386.0
Idx:   1 | reward: 2386.00 | Coverage Progress: 0.6957 |Done: False
[2045    0    3]
2015.0
Idx:   2 | reward: 2015.00 | Coverage Progress: 0.9002 |Done: False
[224   0   4]
184.0
Idx:   3 | reward:  184.00 | Coverage Progress: 0.9226 |Done: False
[49  0  5]
-1.0
Idx:   4 | reward:   -1.00 | Coverage Progress: 0.9275 |Done: False
[320   0   6]
260.0
Idx:   5 | reward:  260.00 | Coverage Progress: 0.9595 |Done: False
[5 0 7]
-65.0
Idx:   6 | reward:  -65.00 | Coverage Progress: 0.9600 |Done: False
[19  0  8]
-61.0
Idx:   7 | reward:  -61.00 | Coverage Progress: 0.9619 |Done: False
[278   1   9]
288.0
Idx:   8 | reward:  288.00 | Coverage Progress: 0.9897 |Done: True


/tmp/ipykernel_129734/2393501048.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action = torch.tensor(action, dtype=torch.float32).cuda()


In [4]:
actions = env.actions

vis_mesh = o3d.geometry.TriangleMesh()
vis_mesh.vertices = o3d.utility.Vector3dVector(env.vertices.cpu().numpy())
vis_mesh.triangles = o3d.utility.Vector3iVector(env.faces.cpu().numpy())

objects = [vis_mesh]

radius = env.radius

for action in actions:
    # Create a sphere to represent the viewpoint
    viewpoint = o3d.geometry.TriangleMesh.create_sphere(radius=0.5)
    viewpoint.compute_vertex_normals()
    viewpoint.paint_uniform_color([1, 0, 0])  # Blue color for viewpoint

    # Set the position of the viewpoint
    # Move last_action to CPU before using with numpy
    action_cpu = action.cpu().numpy()
    theta = (action_cpu[0] + 1) * np.pi
    phi = action_cpu[1] * np.pi / 2
    x = radius * np.sin(theta) * np.cos(phi)
    y = radius * np.sin(theta) * np.sin(phi)
    z = radius * np.cos(theta)
    viewpoint.translate([x, y, z])

    objects.append(viewpoint)

o3d.visualization.draw_geometries(objects, mesh_show_wireframe = False)

## Training a Sample Agent

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

2024-09-30 02:15:08.429597: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 02:15:08.431276: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-30 02:15:08.459192: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 02:15:08.941856: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
env = ObjCoverageEnv("/home/dir/RL_CoveragePlanning/test_models/modified/test_7.obj")

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=int(1e5), progress_bar=True)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.8     |
|    ep_rew_mean     | 1e+03    |
| time/              |          |
|    fps             | 1301     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 18.6         |
|    ep_rew_mean          | 1e+03        |
| time/                   |              |
|    fps                  | 1057         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0066525214 |
|    clip_fraction        | 0.035        |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.83        |
|    explained_variance   | -0.000432    |
|    learning_rate        | 0.0003       |
|    loss                 | 4.36e+04     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00249     |
|    std                  | 0.99         |
|    value_loss           | 6.07e+04     |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 20           |
|    ep_rew_mean          | 1e+03        |
| time/                   |              |
|    fps                  | 1017         |
|    iterations           | 3            |
|    time_elapsed         | 6            |
|    total_timesteps      | 6144         |
| train/                  |              |
|    approx_kl            | 0.0035286718 |
|    clip_fraction        | 0.0213       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.81        |
|    explained_variance   | -0.0108      |
|    learning_rate        | 0.0003       |
|    loss                 | 4.13e+04     |
|    n_updates            | 20           |
|    policy_gradient_loss | -0.00164     |
|    std                  | 0.984        |
|    value_loss           | 6.34e+04     |
------------------------------------------


...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 48.6       |
|    ep_rew_mean          | 1e+03      |
| time/                   |            |
|    fps                  | 949        |
|    iterations           | 45         |
|    time_elapsed         | 97         |
|    total_timesteps      | 92160      |
| train/                  |            |
|    approx_kl            | 0.00410219 |
|    clip_fraction        | 0.0699     |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.73      |
|    explained_variance   | 5.96e-08   |
|    learning_rate        | 0.0003     |
|    loss                 | 2.41e+04   |
|    n_updates            | 440        |
|    policy_gradient_loss | -0.000667  |
|    std                  | 0.946      |
|    value_loss           | 2.3e+04    |
----------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 54.7         |
|    ep_rew_mean          | 1e+03        |
| time/                   |              |
|    fps                  | 949          |
|    iterations           | 46           |
|    time_elapsed         | 99           |
|    total_timesteps      | 94208        |
| train/                  |              |
|    approx_kl            | 0.0041794716 |
|    clip_fraction        | 0.0495       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.72        |
|    explained_variance   | -2.38e-07    |
|    learning_rate        | 0.0003       |
|    loss                 | 1.1e+04      |
|    n_updates            | 450          |
|    policy_gradient_loss | -0.00111     |
|    std                  | 0.94         |
|    value_loss           | 2.12e+04     |
------------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 61.3        |
|    ep_rew_mean          | 1e+03       |
| time/                   |             |
|    fps                  | 949         |
|    iterations           | 47          |
|    time_elapsed         | 101         |
|    total_timesteps      | 96256       |
| train/                  |             |
|    approx_kl            | 0.004353442 |
|    clip_fraction        | 0.0712      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.71       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 2.87e+03    |
|    n_updates            | 460         |
|    policy_gradient_loss | -0.00182    |
|    std                  | 0.935       |
|    value_loss           | 1.71e+04    |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 71.5        |
|    ep_rew_mean          | 1e+03       |
| time/                   |             |
|    fps                  | 949         |
|    iterations           | 48          |
|    time_elapsed         | 103         |
|    total_timesteps      | 98304       |
| train/                  |             |
|    approx_kl            | 0.006320783 |
|    clip_fraction        | 0.0947      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.71       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 2.79e+03    |
|    n_updates            | 470         |
|    policy_gradient_loss | -0.00203    |
|    std                  | 0.941       |
|    value_loss           | 1.48e+04    |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 84.2        |
|    ep_rew_mean          | 1e+03       |
| time/                   |             |
|    fps                  | 949         |
|    iterations           | 49          |
|    time_elapsed         | 105         |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.005269033 |
|    clip_fraction        | 0.0388      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.71       |
|    explained_variance   | 0           |
|    learning_rate        | 0.0003      |
|    loss                 | 6.52e+03    |
|    n_updates            | 480         |
|    policy_gradient_loss | -0.00318    |
|    std                  | 0.938       |
|    value_loss           | 9.61e+03    |
-----------------------------------------


In [6]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=100)

In [7]:
print(f"{mean_reward=} | {std_reward=}")

mean_reward=774.6999999999998 | std_reward=2.2737367544323206e-13
